In [5]:
import os
import tensorflow as tf
from tensorflow import keras

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [7]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.nn.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()
# 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 回调，自动保存检查点

In [8]:
# 创建点路径
cp_path='training_1/cp.ckpt'
cp_dir=os.path.dirname(cp_path)
# 创建检点回调
cp_callback=tf.keras.callbacks.ModelCheckpoint(cp_path,save_weights_only=True,verbose=1)
# 训练模型
model.fit(train_images,train_labels,epochs=10,validation_data=(test_images,test_labels),callbacks=[cp_callback])
# 默认的 TensorFlow 格式仅保存最近的 5 个检查点

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 6s 6ms/step - loss: 1.2054 - acc: 0.6610 - val_loss: 0.7038 - val_acc: 0.7890

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 178us/step - loss: 0.4401 - acc: 0.8790 - val_loss: 0.5313 - val_acc: 0.8450

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 227us/step - loss: 0.3097 - acc: 0.9120 - val_loss: 0.4563 - val_acc: 0.8560

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 184us/step - loss: 0.2105 - acc: 0.9470 - val_loss: 0.4419 - val_acc: 0.8560

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 180us/step - loss: 0.1486 - acc: 0.9720 - val_loss: 0.4114 - val_acc: 0.8650

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=======

#### 创建新模型并进行评估

In [9]:
# 创建一个新模型
model_new=create_model()
# 评估该新模型
loss,acc=model_new.evaluate(test_images,test_labels)
print('untrained model acc{:.2%}'.format(acc))

1000/1000 [==============================] - 0s 148us/step
untrained model acc6.50%


#### 新模型-->加载权重-->进行评估

In [10]:
def create_model_and_evaluate(cp_path): 
    '''
    创建一个新模型，并对新模型进行评估
    '''
    # 创建一个新模型
    model_new=create_model()
    # 从检查点加载权重
    model_new.load_weights(cp_path)
    # 评估该模型
    loss,acc=model_new.evaluate(test_images,test_labels)
    print('restored model accuracy: {:.3%}'.format(acc))

In [11]:
# 自动保存检查点

# 创建点路径
cp_path_id='traing_2/cp-{epoch:04d}.ckpt'
cp_dir_id=os.path.dirname(cp_path_id)
# 创建检点回调
cp_callback_id=tf.keras.callbacks.ModelCheckpoint(cp_path_id,verbose=1,save_weights_only=True,period=5)

model_id=create_model()
model_id.fit(train_images,train_labels,epochs=50,callbacks=[cp_callback_id],validation_data=(test_images,test_labels),verbose=0)
# 默认的 TensorFlow 格式仅保存最近的 5 个检查点


Epoch 00005: saving model to traing_2/cp-0005.ckpt

Epoch 00010: saving model to traing_2/cp-0010.ckpt

Epoch 00015: saving model to traing_2/cp-0015.ckpt

Epoch 00020: saving model to traing_2/cp-0020.ckpt

Epoch 00025: saving model to traing_2/cp-0025.ckpt

Epoch 00030: saving model to traing_2/cp-0030.ckpt

Epoch 00035: saving model to traing_2/cp-0035.ckpt

Epoch 00040: saving model to traing_2/cp-0040.ckpt

Epoch 00045: saving model to traing_2/cp-0045.ckpt

Epoch 00050: saving model to traing_2/cp-0050.ckpt


#### 查看最新的检查点

In [12]:
# 选择最新的检查点
last = tf.train.latest_checkpoint(cp_dir_id)
last

'traing_2\\cp-0050.ckpt'

#### 手动保存检查点

In [13]:
# 路径
cp_path_hand='./hand_save_cp/my_cp'
# 手动进行保存检查点
model_id.save_weights(cp_path_hand)
# 创建一个新模型，并对新模型进行评估
create_model_and_evaluate(cp_path_hand)

1000/1000 [==============================] - 0s 105us/step
restored model accuracy: 87.500%


### 保存整个模型(HDF5标准格式)
保存的内容：
- 权重值
- 模型配置(架构)
- 优化器配置

In [14]:
model =create_model()
model.fit(train_images,train_labels,epochs=5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 483us/step - loss: 1.1736 - acc: 0.6870
Epoch 2/5
1000/1000 [==============================] - 0s 150us/step - loss: 0.4371 - acc: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 125us/step - loss: 0.2887 - acc: 0.9160
Epoch 4/5
1000/1000 [==============================] - 0s 140us/step - loss: 0.2071 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 131us/step - loss: 0.1590 - acc: 0.9650
